# Desenvolver o codigo para automatizar a busca no Rec Aq

* https://www.reclameaqui.com.br/empresa/bet365/lista-reclamacoes/?pagina=1

In [2]:
from bs4 import BeautifulSoup
from unidecode import unidecode

import pandas as pd
import numpy as np

import string
import requests

In [7]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

def get_complains(company_id, company_name):

    complains = {'company_name': [], 'description': [], 'href': [], 'full_complain': []}
    
    for n in range(0, 1001, 10):
        url = f'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{n}'
        res = requests.get(url, params={'company': company_id}, headers=HEADERS).json()

        for x in range(10):
            data = res['complainResult']['complains']['data']
            
            description = data[x]['description']
            soup = BeautifulSoup(description, 'lxml').text
            complains['description'].append(soup)
            
            id_ = data[x]['id']
            title = data[x]['title']
            clean_title = title.translate(str.maketrans('', '', string.punctuation))
            href = clean_title.lower().strip() + '_' + id_
            href = unidecode(href.replace(' ', '-'))
            complains['href'].append(href)
            
            complains['company_name'].append(company_name)
            
            complains['full_complain'].append('')
            
    df = pd.DataFrame(complains)
    
    return df

In [8]:
%%time

df = get_complains(76804, 'bet365')

df.head()

/home/vitor/.pyenv/versions/3.10.6/envs/estudo/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


CPU times: user 7.95 s, sys: 72.1 ms, total: 8.03 s
Wall time: 10.8 s


,company_name,description,href,full_complain
0,bet365,"Eu criei a conta num dia , usei , depositei di...",exclusao-da-conta-do-nada_Dud9R8-AUMYcE5wj,
1,bet365,Fiz o cadastro na bet365 coloquei 100 reais e ...,quero-uma-solucao-o-mais-rapid9-possivel_zsCsP...,
2,bet365,"Possuo um cadastro com saldo no site, não poss...",exclusao-de-cadastro_GvIGnA-pGALXPADB,
3,bet365,Bom dia quero fazer aposta porém não validara...,validade-de-indentidade_9Aa2wovUP31RRsuK,
4,bet365,Eu fiz um depósito de 30 reais na bet 365 e c...,a-bet-tirou-meu-dinheiro_DGK8-zT7j9HCgFW-,


In [11]:
%%time

for i, company_name, href in zip(df.index, df.company_name, df.href):
    try:
        url = f"https://www.reclameaqui.com.br/{company_name}/{href}/"
        res = requests.get(url, headers=HEADERS)

        soup = BeautifulSoup(res.text, 'lxml')
        complain = soup.find('p', {'class': 'lzlu7c-17 fXwQIB'}).text
        df.iloc[i]['full_complain'] = complain
    except:
        print(href)
        df.drop(index=i, inplace=True)

df.head()

CPU times: user 2min 3s, sys: 962 ms, total: 2min 4s
Wall time: 22min 27s


,company_name,description,href,full_complain
0,bet365,"Eu criei a conta num dia , usei , depositei di...",exclusao-da-conta-do-nada_Dud9R8-AUMYcE5wj,"Eu criei a conta num dia , usei , depositei di..."
1,bet365,Fiz o cadastro na bet365 coloquei 100 reais e ...,quero-uma-solucao-o-mais-rapid9-possivel_zsCsP...,Fiz o cadastro na bet365 coloquei 100 reais e ...
2,bet365,"Possuo um cadastro com saldo no site, não poss...",exclusao-de-cadastro_GvIGnA-pGALXPADB,"Possuo um cadastro com saldo no site, não poss..."
3,bet365,Bom dia quero fazer aposta porém não validara...,validade-de-indentidade_9Aa2wovUP31RRsuK,Bom dia quero fazer aposta porém não validara...
4,bet365,Eu fiz um depósito de 30 reais na bet 365 e c...,a-bet-tirou-meu-dinheiro_DGK8-zT7j9HCgFW-,Eu fiz um depósito de 30 reais na bet 365 e c...
